# Manejo de Errores al Cargar Archivos JSON en Notebook Architect

En este notebook, aprenderemos a manejar errores comunes al cargar archivos JSON malformados en Notebook Architect. Los errores de formato JSON son muy comunes y pueden ser difíciles de depurar sin la información adecuada.

## Objetivos
- Simular problemas de carga con JSON malformado
- Capturar y mostrar errores de manera amigable
- Implementar validación previa en JavaScript
- Mostrar soluciones para corregir archivos JSON incorrectos

## 1. Simulación de Carga de Archivo JSON Malformado

Cuando cargamos un archivo JSON en Notebook Architect, se espera que este tenga un formato válido según la especificación JSON. Sin embargo, es común encontrar archivos con errores de sintaxis como:

- Falta de comillas en las claves
- Comas faltantes o sobrantes
- Llaves o corchetes sin cerrar
- Valores sin delimitadores correctos

Vamos a simular un archivo JSON con errores para ver cómo se comporta el sistema.

In [ ]:
# Crear un ejemplo de JSON malformado
json_malformado = """{
  "nombre" "Juan Pérez",
  "edad": 30,
  "ciudad": "Madrid",
  "intereses": ["programación", "datos", "visualización"]
}"""

print("JSON malformado:")
print(json_malformado)

# El error está en la línea 2, columna 5 (falta el caracter ':' después de "nombre")
print("\nError en este JSON: falta el caracter ':' después de \"nombre\" en la línea 2")

## 2. Manejo de Errores al Cargar JSON con Pyodide

Cuando intentamos cargar un JSON malformado usando pandas a través de Pyodide, obtenemos un error similar al que se muestra en la consola del navegador. Vamos a intentar cargar el JSON malformado y capturar el error para mostrar un mensaje más amigable.

In [ ]:
import pandas as pd
import json
import traceback

# Intentar cargar el JSON malformado y capturar el error
try:
    # Primero intentamos parsear el JSON
    datos_json = json.loads(json_malformado)
    
    # Si no hay error, convertimos a DataFrame de pandas
    df = pd.DataFrame([datos_json])
    print("JSON cargado exitosamente:")
    print(df)
    
except json.JSONDecodeError as e:
    # Capturar el error específico de decodificación JSON
    error_message = f"Error de formato JSON: {str(e)}"
    error_linea = e.lineno
    error_columna = e.colno
    error_pos = e.pos
    
    # Mostrar información detallada del error
    print("❌ " + error_message)
    print(f"📍 Línea: {error_linea}, Columna: {error_columna}, Posición: {error_pos}")
    
    # Mostrar las líneas cercanas al error para ayudar a identificarlo
    lineas = json_malformado.split('\n')
    inicio = max(0, error_linea - 2)
    fin = min(len(lineas), error_linea + 1)
    
    print("\nContexto del error:")
    for i, linea in enumerate(lineas[inicio:fin], inicio + 1):
        marcador = "➤ " if i == error_linea else "  "
        print(f"{marcador}{i}: {linea}")
        
        # Agregar indicador visual de la posición exacta del error
        if i == error_linea:
            espacios = " " * (len(f"{marcador}{i}: ") + error_columna - 1)
            print(f"{espacios}^-- Error aquí")
    
    # Mostrar el traceback completo para depuración
    print("\nTraceback completo:")
    traceback.print_exc()
    
except Exception as e:
    # Capturar cualquier otro error
    print(f"Error inesperado: {str(e)}")
    traceback.print_exc()

## 3. Visualización del Mensaje de Error en el Notebook

Los mensajes de error pueden ser difíciles de interpretar para los usuarios no técnicos. A continuación, vamos a crear una función que genere mensajes de error más amigables y visuales, con sugerencias para corregir problemas comunes de formato JSON.

In [ ]:
from IPython.display import HTML, display
import re

def mostrar_error_json_amigable(json_texto, error):
    """
    Muestra un mensaje de error JSON amigable con formato HTML
    """
    # Extraer información del error
    mensaje = str(error)
    
    # Detectar el tipo de error común en JSON
    tipo_error = "desconocido"
    sugerencia = ""
    
    if "Expecting ':'" in mensaje:
        tipo_error = "Falta un delimitador ':'"
        sugerencia = "Asegúrate de que todas las claves (nombres de propiedades) tengan el caracter ':' después del nombre."
    elif "Expecting ',' delimiter" in mensaje:
        tipo_error = "Falta una coma ','"
        sugerencia = "Revisa que todos los elementos estén separados por comas."
    elif "Expecting property name" in mensaje:
        tipo_error = "Nombre de propiedad incorrecto"
        sugerencia = "Verifica que todas las propiedades tengan comillas dobles."
    elif "Expecting value" in mensaje:
        tipo_error = "Valor incorrecto"
        sugerencia = "Hay un valor con formato incorrecto o falta un valor."
    elif "Extra data" in mensaje:
        tipo_error = "Datos adicionales"
        sugerencia = "Hay contenido adicional después del JSON válido."
    
    # Obtener línea y columna del error
    linea_match = re.search(r'line (\d+)', mensaje)
    columna_match = re.search(r'column (\d+)', mensaje)
    
    if linea_match and columna_match:
        linea = int(linea_match.group(1))
        columna = int(columna_match.group(1))
        
        # Extraer las líneas para mostrar el contexto
        lineas = json_texto.split('\n')
        inicio = max(0, linea - 2)
        fin = min(len(lineas), linea + 1)
        
        # Crear el HTML para mostrar el error
        html = f"""
        <div style="background-color: #fff3f3; padding: 15px; border-left: 5px solid #ff5252; margin: 10px 0;">
            <h3 style="color: #d32f2f; margin-top: 0;">Error de formato JSON</h3>
            <p><strong>Problema:</strong> {tipo_error}</p>
            <p><strong>Ubicación:</strong> Línea {linea}, Columna {columna}</p>
            <p><strong>Sugerencia:</strong> {sugerencia}</p>
            
            <div style="background-color: #f5f5f5; padding: 10px; font-family: monospace; white-space: pre; margin-top: 10px;">
        """
        
        # Agregar las líneas de contexto
        for i, linea_texto in enumerate(lineas[inicio:fin], inicio + 1):
            if i == linea:
                # Resaltar la línea con el error
                html += f'<div style="background-color: #ffebee;"><span style="color: #777;">{i}:</span> {linea_texto[:columna-1]}<span style="color: red; font-weight: bold; text-decoration: underline;">{linea_texto[columna-1:columna]}</span>{linea_texto[columna:]}</div>'
                # Agregar marcador de posición
                marcador = ' ' * (len(str(i)) + 2 + columna - 1)
                html += f'<div style="color: #d32f2f;">{marcador}^-- Error aquí</div>'
            else:
                html += f'<div><span style="color: #777;">{i}:</span> {linea_texto}</div>'
        
        html += """
            </div>
            <p style="margin-top: 10px;"><strong>Acción recomendada:</strong> Corrige el formato JSON según la sugerencia y vuelve a intentar cargar el archivo.</p>
        </div>
        """
        
        # Mostrar el HTML
        display(HTML(html))
    else:
        # Si no se puede extraer línea y columna, mostrar el mensaje original
        display(HTML(f"""
        <div style="background-color: #fff3f3; padding: 15px; border-left: 5px solid #ff5252; margin: 10px 0;">
            <h3 style="color: #d32f2f; margin-top: 0;">Error de formato JSON</h3>
            <p>{mensaje}</p>
        </div>
        """))

# Probar la función con nuestro JSON malformado
try:
    datos = json.loads(json_malformado)
except json.JSONDecodeError as e:
    mostrar_error_json_amigable(json_malformado, e)

## 4. Validación Previa del JSON antes de Enviar a Pyodide

Para evitar que errores de formato JSON lleguen hasta Pyodide, podemos implementar una validación previa en JavaScript. Esto ahorra tiempo y recursos, ya que detectamos el problema antes de iniciar el procesamiento en Python.

A continuación, mostramos un ejemplo de cómo implementar esta validación en JavaScript, que podría integrarse en el código de `data-loader.js` de Notebook Architect.

In [ ]:
// Función para validar JSON y mostrar errores detallados
function validarJSON(jsonString) {
    try {
        // Intentar parsear el JSON
        JSON.parse(jsonString);
        return { 
            valido: true, 
            mensaje: "JSON válido" 
        };
    } catch (error) {
        // Extraer información útil del error
        const mensaje = error.message;
        
        // Patrones comunes para extraer línea y posición
        const posicionMatch = mensaje.match(/position (\d+)/);
        const posicion = posicionMatch ? parseInt(posicionMatch[1]) : -1;
        
        // Calcular línea y columna basado en la posición
        let linea = 1;
        let columna = 0;
        let contadorPos = 0;
        
        if (posicion >= 0) {
            for (let i = 0; i < posicion && i < jsonString.length; i++) {
                contadorPos++;
                if (jsonString[i] === '\n') {
                    linea++;
                    columna = 0;
                } else {
                    columna++;
                }
            }
        }
        
        // Extraer contexto alrededor del error
        let contexto = "";
        if (posicion >= 0) {
            const lineas = jsonString.split('\n');
            const lineaActual = linea - 1;
            const inicio = Math.max(0, lineaActual - 2);
            const fin = Math.min(lineas.length, lineaActual + 2);
            
            for (let i = inicio; i < fin; i++) {
                const lineaNumero = i + 1;
                const marcador = lineaNumero === linea ? "➤ " : "  ";
                contexto += `${marcador}${lineaNumero}: ${lineas[i]}\n`;
                
                // Agregar indicador visual de la posición exacta del error
                if (lineaNumero === linea) {
                    const espacios = " ".repeat(columna + marcador.length + lineaNumero.toString().length + 2);
                    contexto += `${espacios}^ Error aquí\n`;
                }
            }
        }
        
        // Generar sugerencias basadas en el tipo de error
        let sugerencia = "Revisa la sintaxis JSON.";
        if (mensaje.includes("Unexpected token")) {
            sugerencia = "Hay un carácter inesperado. Verifica que todas las comillas, comas y llaves estén correctamente utilizadas.";
        } else if (mensaje.includes("Unexpected end of JSON")) {
            sugerencia = "El JSON está incompleto. Verifica que todas las llaves o corchetes estén cerrados.";
        } else if (mensaje.includes("Expected property name")) {
            sugerencia = "Falta un nombre de propiedad o no está entre comillas.";
        } else if (mensaje.includes("Expected ':'")) {
            sugerencia = "Falta un delimitador ':' después de un nombre de propiedad.";
        } else if (mensaje.includes("Expected ','")) {
            sugerencia = "Falta una coma ',' separando elementos.";
        }
        
        return {
            valido: false,
            mensaje: `Error de formato JSON: ${mensaje}`,
            linea,
            columna,
            posicion,
            contexto,
            sugerencia,
            error
        };
    }
}

// Ejemplo de implementación en data-loader.js
function loadJSONData(jsonContent, datasetName) {
    // Validar el JSON antes de enviarlo a Pyodide
    const validacion = validarJSON(jsonContent);
    
    if (!validacion.valido) {
        console.error('Error en formato JSON:', validacion.mensaje);
        console.log('Contexto del error:\n', validacion.contexto);
        console.log('Sugerencia:', validacion.sugerencia);
        
        // En lugar de continuar con la carga, mostrar un error al usuario
        throw new Error(`El archivo JSON tiene un formato inválido (línea ${validacion.linea}, columna ${validacion.columna}): ${validacion.sugerencia}`);
    }
    
    // Si el JSON es válido, continuar con el procesamiento
    console.log('JSON válido, continuando con la carga en Pyodide...');
    
    // Aquí continuaría el código original para cargar el JSON en Pyodide
}

// Probar la función con nuestro JSON malformado
const jsonMalformado = `{
  "nombre" "Juan Pérez",
  "edad": 30,
  "ciudad": "Madrid",
  "intereses": ["programación", "datos", "visualización"]
}`;

const resultado = validarJSON(jsonMalformado);
console.log("Resultado de la validación:", resultado);

## 5. Solución: JSON Corregido

Ahora que hemos identificado y entendido el error, podemos corregir el JSON y cargarlo correctamente. El problema estaba en la falta del caracter ':' después de la propiedad "nombre".

In [ ]:
# JSON corregido
json_corregido = """{
  "nombre": "Juan Pérez",
  "edad": 30,
  "ciudad": "Madrid",
  "intereses": ["programación", "datos", "visualización"]
}"""

print("JSON corregido:")
print(json_corregido)

# Cargar el JSON corregido
import pandas as pd
import json

try:
    # Parsear el JSON
    datos_json = json.loads(json_corregido)
    
    # Convertir a DataFrame de pandas
    df = pd.DataFrame([datos_json])
    print("\nJSON cargado exitosamente:")
    display(df)
    
    # También podemos normalizar los datos para manejar listas anidadas
    df_normalizado = pd.json_normalize(datos_json)
    print("\nJSON normalizado (mejor para estructuras anidadas):")
    display(df_normalizado)
    
except json.JSONDecodeError as e:
    print(f"Error de formato JSON: {str(e)}")
except Exception as e:
    print(f"Error inesperado: {str(e)}")

## Conclusiones y Recomendaciones

En este notebook hemos aprendido a:

1. **Identificar errores comunes** en archivos JSON malformados
2. **Capturar y presentar errores** de manera amigable y útil para el usuario
3. **Implementar validación previa** en JavaScript antes de enviar datos a Pyodide
4. **Corregir archivos JSON** basándonos en los mensajes de error

### Implementación en Notebook Architect

Para implementar estas mejoras en el sistema Notebook Architect, se recomienda:

1. Agregar la función de validación JavaScript en `data-loader.js` antes de enviar cualquier JSON a Pyodide
2. Mejorar los mensajes de error para mostrar información contextual sobre dónde ocurrió el error
3. Proporcionar sugerencias específicas según el tipo de error encontrado
4. Considerar la posibilidad de agregar una herramienta de corrección automática para problemas comunes

### Próximos Pasos

- Implementar validación para otros formatos de datos (CSV, Excel)
- Agregar herramientas para detectar y corregir automáticamente problemas comunes
- Mejorar la visualización de errores en la interfaz de usuario
- Crear documentación para usuarios sobre formatos de datos correctos